In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import time
import re

<IPython.core.display.Javascript object>

In [3]:
# load in scraped tweets from https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Web_Scrape.ipynb
tweets = pd.read_csv(
    r"/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/text-query-tweets.csv"
)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,1,10,11,12,13,19,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<IPython.core.display.Javascript object>

In [4]:
tweets.head()

,Unnamed: 0,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,0,0,https://twitter.com/LewisHowes/status/13513399...,2021-01-19 01:25:18+00:00,@Arashdeep2002 Exactly! 💯,@Arashdeep2002 Exactly! 💯,1.351340e+18,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],...,0,1.167437e+18,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'Arashdeep2002', 'displayname': ..."
1,1,1,https://twitter.com/LewisHowes/status/13513130...,2021-01-18 23:38:31+00:00,@EDGcoach Thank you!,@EDGcoach Thank you!,1.351313e+18,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],...,0,1.350508e+18,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'EDGcoach', 'displayname': 'Empo..."
2,2,2,https://twitter.com/LewisHowes/status/13513042...,2021-01-18 23:03:45+00:00,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,1.351304e+18,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],...,0,1.350102e+18,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'ogony_john', 'displayname': 'OG..."
3,3,3,https://twitter.com/LewisHowes/status/13512669...,2021-01-18 20:35:31+00:00,I hope this speaks to you today 🙏 https://t.co...,I hope this speaks to you today 🙏 https://t.co...,1.351267e+18,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],...,8,1.351267e+18,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,[{'previewUrl': 'https://pbs.twimg.com/media/E...,NaN,NaN,NaN
4,4,4,https://twitter.com/LewisHowes/status/13511821...,2021-01-18 14:58:29+00:00,@reubenchavez9 Thank you for sharing your thou...,@reubenchavez9 Thank you for sharing your thou...,1.351182e+18,"{'username': 'LewisHowes', 'displayname': 'Lew...",[],[],...,0,1.350102e+18,en,"<a href=""http://www.zendesk.com"" rel=""nofollow...",http://www.zendesk.com,Zendesk,NaN,NaN,NaN,"[{'username': 'reubenchavez9', 'displayname': ..."


<IPython.core.display.Javascript object>

In [5]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977472 entries, 0 to 977471
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       977472 non-null  object 
 1   index            977472 non-null  object 
 2   url              977465 non-null  object 
 3   date             977465 non-null  object 
 4   content          977465 non-null  object 
 5   renderedContent  977458 non-null  object 
 6   id               977458 non-null  float64
 7   user             977458 non-null  object 
 8   outlinks         977458 non-null  object 
 9   tcooutlinks      977458 non-null  object 
 10  replyCount       977458 non-null  object 
 11  retweetCount     977458 non-null  object 
 12  likeCount        977458 non-null  object 
 13  quoteCount       977458 non-null  object 
 14  conversationId   977451 non-null  float64
 15  lang             977451 non-null  object 
 16  source           977450 non-null  obje

<IPython.core.display.Javascript object>

In [6]:
tweets.applymap(type)

,Unnamed: 0,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>
1,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>
2,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>
3,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>
4,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977467,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
977468,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
977469,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>
977470,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,...,<class 'int'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'float'>


<IPython.core.display.Javascript object>

In [7]:
# extend max col width so all the text can be seen
pd.set_option("display.max_colwidth", None)

tweets["content"][:20]

0                                     @Arashdeep2002 Exactly! 💯
1                                          @EDGcoach Thank you!
2             @ogony_john Indeed! Thanks a lot for the tweet! 🙌
3     I hope this speaks to you today 🙏 https://t.co/heACOl1v22
4        @reubenchavez9 Thank you for sharing your thoughts!! ❤
5                                     @ninachristian Exactly! 💯
6                                        @JoshConstine Love it!
7                                             @james_erice Yes!
8                               @JasonRomano Indeed, thank you!
9                         @CoachJimJohnson Thank you so much! 🙌
10                                   @mattgmorris_ Thank you! 🙌
11                                       @LoriMoreno Thank you!
12                                      @JamelleSanders Indeed!
13                                       @rod_bahadosingh Yes 💯
14                                @thehappyhustle Absolutely! 💯
15                                    @n

<IPython.core.display.Javascript object>

In [8]:
tweets["content"][:-10]

0                                                                                                                @Arashdeep2002 Exactly! 💯
1                                                                                                                     @EDGcoach Thank you!
2                                                                                        @ogony_john Indeed! Thanks a lot for the tweet! 🙌
3                                                                                I hope this speaks to you today 🙏 https://t.co/heACOl1v22
4                                                                                   @reubenchavez9 Thank you for sharing your thoughts!! ❤
                                                                        ...                                                               
977457    Make big plans; aim high in hope and work, remembering that a noble, logical diagram once recorded will not die.\nDaniel Burnham
977458                     

<IPython.core.display.Javascript object>

After further inspection of the content column, tweets that start with @ are specific tweets to other user. These tweets will be removed becuase we want motinational tweets that are geared towards a general audience. We also have to remove tweets that have hyperlinks since those tweets are more calls to action and not general motivational advise. We will also have to deal with emojis.  

In [9]:
# convert any non-text to text in the content column
tweets["content"] = tweets["content"].map(str)

<IPython.core.display.Javascript object>

In [10]:
# remove all tweets that contain @
tweets2 = tweets[~tweets['content'].str.contains("@")]


<IPython.core.display.Javascript object>

In [11]:
tweets2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 796080 entries, 3 to 977471
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       796080 non-null  object 
 1   index            796080 non-null  object 
 2   url              796073 non-null  object 
 3   date             796073 non-null  object 
 4   content          796080 non-null  object 
 5   renderedContent  796067 non-null  object 
 6   id               796067 non-null  float64
 7   user             796067 non-null  object 
 8   outlinks         796067 non-null  object 
 9   tcooutlinks      796067 non-null  object 
 10  replyCount       796067 non-null  object 
 11  retweetCount     796067 non-null  object 
 12  likeCount        796067 non-null  object 
 13  quoteCount       796067 non-null  object 
 14  conversationId   796060 non-null  float64
 15  lang             796060 non-null  object 
 16  source           796059 non-null  obje

<IPython.core.display.Javascript object>

That removed about 180,000 rows. 

In [12]:
tweets2["content"][:20]

3                                                                                                                                                                                                                                                              I hope this speaks to you today 🙏 https://t.co/heACOl1v22
28                          Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:\n\n1. Look at what’s not serving you\n2. Listen to your intuition\n3. Learn the meaning behind it\n4. Let go of limits that hold you back
44                                CALLING ALL HIGH ACHIEVERS:\n\nIf you are an entrepreneur, brand, or business owner ready to invest in a premium high-performance coaching program...\n\nClick here: https://t.co/cQuLdxqiTH if you want to put yourself on an accelerated path to greatness.🔥 https://t.co/IgwaulAlX5
45                                 Protect your inner peace a

<IPython.core.display.Javascript object>

In [13]:
# drop rows containing http links
tweets3 = tweets2[~(tweets2["content"].str.contains("http"))]

<IPython.core.display.Javascript object>

In [14]:
tweets3["content"][:20]

28      Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:\n\n1. Look at what’s not serving you\n2. Listen to your intuition\n3. Learn the meaning behind it\n4. Let go of limits that hold you back
45             Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn’t consume you. You don’t need to be someone else’s emotional punching bag. Your energy and happiness is too important. Protect your peace!
92                You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn’t wish it on anyone else. But I also know you wouldn’t change a thing!
145                                       Always remember to ask for exactly what you want. Ask for love, good health, ab

<IPython.core.display.Javascript object>

In [15]:
len(tweets3)

697618

<IPython.core.display.Javascript object>

We have removed about a quarter of our initial database. We still need to remove line breaks, non-english text, emojis, and emoticons.

In [16]:
tweets3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 697618 entries, 28 to 977471
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       697618 non-null  object 
 1   index            697618 non-null  object 
 2   url              697611 non-null  object 
 3   date             697611 non-null  object 
 4   content          697618 non-null  object 
 5   renderedContent  697605 non-null  object 
 6   id               697605 non-null  float64
 7   user             697605 non-null  object 
 8   outlinks         697605 non-null  object 
 9   tcooutlinks      697605 non-null  object 
 10  replyCount       697605 non-null  object 
 11  retweetCount     697605 non-null  object 
 12  likeCount        697605 non-null  object 
 13  quoteCount       697605 non-null  object 
 14  conversationId   697603 non-null  float64
 15  lang             697603 non-null  object 
 16  source           697603 non-null  obj

<IPython.core.display.Javascript object>

In [17]:
# remove line breaks and change non-latin apostrophe to latin counterpart
tweets3["content"] = tweets3["content"].str.replace("\n", " ").str.replace("’", "'")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<IPython.core.display.Javascript object>

In [18]:
tweets3["content"][:20]

28         Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:  1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back
45           Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!
92              You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn't wish it on anyone else. But I also know you wouldn't change a thing!
145                                     Always remember to ask for exactly what you want. Ask for love, good health, abundance 

<IPython.core.display.Javascript object>

Next we will remove text that contains more than one character outside of the latin alphabet. This will include emojis and emoticons by taking advantage of the ord() function. This step won't completely remove all non-english tweets, but it will deal with emojis and emoticons. Languages like French, German, and Spanish use a latin alphabet. We will remove the rest of the non-english text later. 

In [19]:
# function that returns True if no more than one character is non-latin
def is_latin_alphabet(string):
    non_ascii = 0

    for character in string:
        if ord(character) > 127:
            non_ascii += 1

    if non_ascii > 1:
        return False
    else:
        return True

<IPython.core.display.Javascript object>

In [20]:
# make a dictionary with emoji less text or append with none
latin_emojiless_text = []

for text in tweets3["content"]:
    if is_latin_alphabet(text) == True:
        latin_emojiless_text.append(text)
    else:
        latin_emojiless_text.append(None)


len(latin_emojiless_text)

697618

<IPython.core.display.Javascript object>

In [21]:
# save emojiless text to new dataframe
tweets4 = tweets3.copy()
tweets4["content"] = latin_emojiless_text

<IPython.core.display.Javascript object>

In [22]:
tweets4.head()

,Unnamed: 0,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
28,28,28,https://twitter.com/LewisHowes/status/1350929208520929282,2021-01-17 22:13:17+00:00,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you: 1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:\n\n1. Look at what’s not serving you\n2. Listen to your intuition\n3. Learn the meaning behind it\n4. Let go of limits that hold you back,1.350929e+18,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],...,9,1.350929e+18,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
45,45,45,https://twitter.com/LewisHowes/status/1350102468126883842,2021-01-15 15:28:06+00:00,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn’t consume you. You don’t need to be someone else’s emotional punching bag. Your energy and happiness is too important. Protect your peace!,1.350102e+18,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],...,13,1.350102e+18,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
92,92,92,https://twitter.com/LewisHowes/status/1349033591230001153,2021-01-12 16:40:46+00:00,You are stronger

<IPython.core.display.Javascript object>

In [23]:
tweets4["content"][:-20]

28        Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:  1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back
45          Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!
92             You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn't wish it on anyone else. But I also know you wouldn't change a thing!
145                                    Always remember to ask for exactly what you want. Ask for love, good health, abundance and 

<IPython.core.display.Javascript object>

We do see we will have to remove hashtage as well from our tweets. 

In [24]:
def remove_hashtags(text):
    text = re.sub("#\S+", "", str(text))
    return text

<IPython.core.display.Javascript object>

In [25]:
tweets5 = tweets4.copy()

no_hashtags = [remove_hashtags(text) for text in tweets5["content"]]

<IPython.core.display.Javascript object>

In [26]:
tweets5["content"] = no_hashtags

tweets5["content"][:-20]

28        Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:  1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back
45          Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!
92             You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn't wish it on anyone else. But I also know you wouldn't change a thing!
145                                    Always remember to ask for exactly what you want. Ask for love, good health, abundance and 

<IPython.core.display.Javascript object>

In [27]:
tweets5.shape

(697618, 23)

<IPython.core.display.Javascript object>

In [28]:
tweets5["lang"].value_counts()

en     696874
fr        253
tl        236
es         55
de         47
ca         42
und        28
et         17
it         10
vi          8
da          6
ro          6
in          5
no          5
cy          3
pt          3
nl          2
ht          1
cs          1
lt          1
Name: lang, dtype: int64

<IPython.core.display.Javascript object>

In [29]:
# remove non-english tweets
tweets6 = tweets5[tweets["lang"] == "en"]

tweets6["lang"].value_counts()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


en    696874
Name: lang, dtype: int64

<IPython.core.display.Javascript object>

Next, we will remove unnecessary columns for our chatbot. The only columns we need are the content column, a column that identifies the author of the tweet, and the date. 

In [30]:
tweets6.head()

,Unnamed: 0,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
28,28,28,https://twitter.com/LewisHowes/status/1350929208520929282,2021-01-17 22:13:17+00:00,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you: 1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you:\n\n1. Look at what’s not serving you\n2. Listen to your intuition\n3. Learn the meaning behind it\n4. Let go of limits that hold you back,1.350929e+18,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],...,9,1.350929e+18,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
45,45,45,https://twitter.com/LewisHowes/status/1350102468126883842,2021-01-15 15:28:06+00:00,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn’t consume you. You don’t need to be someone else’s emotional punching bag. Your energy and happiness is too important. Protect your peace!,1.350102e+18,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],...,13,1.350102e+18,en,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,NaN,NaN
92,92,92,https://twitter.com/LewisHowes/status/1349033591230001153,2021-01-12 16:40:46+00:00,You are stronger

<IPython.core.display.Javascript object>

In [31]:
# condense dataframe to only the 3 necessary columns
tweets7 = tweets6[["date", "content", "user"]]

tweets7.head()

,date,content,user
28,2021-01-17 22:13:17+00:00,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you: 1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
45,2021-01-15 15:28:06+00:00,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
92,2021-01-12 16:40:46+00:00,You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn't wish it on anyone else. But I also know you wouldn't change a thing!,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
145,2021-01-07 16:00:29+00:00,"Always remember to ask for exactly what you

<IPython.core.display.Javascript object>

In [34]:
# extract username and display names to seperate columns
# user_name = "username"
# display_name = ["displayname"]

# tweets7["user_name"] = [
#     value[user_name] for (key, value) in tweets7["user"].items() if user_name in value
# ]

<IPython.core.display.Javascript object>

In [35]:
tweets7.applymap(type)

,date,content,user
28,<class 'str'>,<class 'str'>,<class 'str'>
45,<class 'str'>,<class 'str'>,<class 'str'>
92,<class 'str'>,<class 'str'>,<class 'str'>
145,<class 'str'>,<class 'str'>,<class 'str'>
190,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...
977467,<class 'str'>,<class 'str'>,<class 'str'>
977468,<class 'str'>,<class 'str'>,<class 'str'>
977469,<class 'str'>,<class 'str'>,<class 'str'>
977470,<class 'str'>,<class 'str'>,<class 'str'>


<IPython.core.display.Javascript object>

In [36]:
from ast import literal_eval


def try_literal_eval(s):
    try:
        return literal_eval(s)
    except ValueError:
        return s

<IPython.core.display.Javascript object>

In [37]:
tweets7["user"].apply(try_literal_eval)

28        {'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowe

<IPython.core.display.Javascript object>

In [38]:
tweets7.applymap(type)

,date,content,user
28,<class 'str'>,<class 'str'>,<class 'str'>
45,<class 'str'>,<class 'str'>,<class 'str'>
92,<class 'str'>,<class 'str'>,<class 'str'>
145,<class 'str'>,<class 'str'>,<class 'str'>
190,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...
977467,<class 'str'>,<class 'str'>,<class 'str'>
977468,<class 'str'>,<class 'str'>,<class 'str'>
977469,<class 'str'>,<class 'str'>,<class 'str'>
977470,<class 'str'>,<class 'str'>,<class 'str'>


<IPython.core.display.Javascript object>

In [55]:
import json

user_dict = []

for row in tweets7["user"]:
    row_dict = json.loads(row)
    user_dict.append(row_dict)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

<IPython.core.display.Javascript object>

In [39]:
tweets7.head()

,date,content,user
28,2021-01-17 22:13:17+00:00,Know this. Everything is happening for a reason and in your favor for the betterment of your future. The pain you feel now will help later if you: 1. Look at what's not serving you 2. Listen to your intuition 3. Learn the meaning behind it 4. Let go of limits that hold you back,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
45,2021-01-15 15:28:06+00:00,Protect your inner peace at all costs. Create daily practices and communicate boundaries to make sure the stress and drama of others doesn't consume you. You don't need to be someone else's emotional punching bag. Your energy and happiness is too important. Protect your peace!,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
92,2021-01-12 16:40:46+00:00,You are stronger than you think. The painful moments that hurt you in the past were meant for you. They broke you down so you could become stronger than ever before. I know it was hard. I know you wouldn't wish it on anyone else. But I also know you wouldn't change a thing!,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}"
145,2021-01-07 16:00:29+00:00,"Always remember to ask for exactly what you

<IPython.core.display.Javascript object>

In [41]:
# extract username and display names to seperate columns
# user_name = ["username"]
# display_name = ["displayname"]

# tweets7["user_name"] = [
#     value[user_name] for (key, value) in tweets7["user"].items() if user_name in value
# ]

<IPython.core.display.Javascript object>

In [42]:
# for key, value in tweets7["user"][:1].items():
#     value.apply(eval)
# #     for k, v in value:
# #         if k == "username":
# #             print(v)

<IPython.core.display.Javascript object>

In [43]:
print(tweets7["user"].values[0])

{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}


<IPython.core.display.Javascript object>

In [44]:
items = list(tweets7["user"].items())
print(items[0][1])

{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}


<IPython.core.display.Javascript object>

In [45]:
# remove cuss words

<IPython.core.display.Javascript object>

In [46]:
# tweet bot will have a drop down for topic and seperate one for twitter user account


<IPython.core.display.Javascript object>

In [56]:
json = pd.read_json(
    r"/Users/tarrantcarter/Dropbox/Data Science/Final_Capstone/motivational_tweets.json",
)

<IPython.core.display.Javascript object>

In [57]:
json.head()

,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,0,https://twitter.com/LewisHowes/status/1351339921307545606,2021-01-19 01:25:18,@Arashdeep2002 Exactly! 💯,@Arashdeep2002 Exactly! 💯,1351339921307545606,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],0,...,0,1167437342069727233,en,"<a href=""http://www.zendesk.com"" rel=""nofollow"">Zendesk</a>",http://www.zendesk.com,Zendesk,None,NaN,None,"[{'username': 'Arashdeep2002', 'displayname': 'Arashdeep Kaur', 'id': 1344935179542908932, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/Arashdeep2002'}]"
1,1,https://twitter.com/LewisHowes/status/1351313050159222786,2021-01-18 23:38:31,@EDGcoach Thank you!,@EDGcoach Thank you!,1351313050159222786,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],1,...,0,1350507769602965505,en,"<a href=""http://www.zendesk.com"" rel=""nofollow"">Zendesk</a>",http://www.zendesk.com,Zendesk,None,NaN,None,"[{'username': 'EDGcoach', 'displayname': 'Empowering Inferno', 'id': 764510880822235136, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/EDGcoach'}]"
2,2,https://twitter.com/LewisHowes/status/1351304297653350400,2021-01-18 23:03:45,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,1351304297653350400,"{'username': 'LewisH

<IPython.core.display.Javascript object>

In [58]:
json.applymap(type)

,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'list'>
1,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'list'>
2,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'list'>
3,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>
4,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977453,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>
977454,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>
977455,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>
977456,<class 'int'>,<class 'str'>,<class 'pandas._libs.tslibs.timestamps.Timestamp'>,<class 'str'>,<class 'str'>,<class 'int'>,<class 'dict'>,<class 'list'>,<class 'list'>,<class 'int'>,...,<class 'int'>,<class 'int'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'float'>,<class 'NoneType'>,<class 'NoneType'>


<IPython.core.display.Javascript object>

In [59]:
json.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 977458 entries, 0 to 977457
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   index            977458 non-null  int64         
 1   url              977458 non-null  object        
 2   date             977458 non-null  datetime64[ns]
 3   content          977458 non-null  object        
 4   renderedContent  977458 non-null  object        
 5   id               977458 non-null  int64         
 6   user             977458 non-null  object        
 7   outlinks         977458 non-null  object        
 8   tcooutlinks      977458 non-null  object        
 9   replyCount       977458 non-null  int64         
 10  retweetCount     977458 non-null  int64         
 11  likeCount        977458 non-null  int64         
 12  quoteCount       977458 non-null  int64         
 13  conversationId   977458 non-null  int64         
 14  lang             977

<IPython.core.display.Javascript object>

In [62]:
# extract username and display names to seperate columns
user_name = "username"
display_name = ["displayname"]

json["user_name"] = [
    value[user_name] for (key, value) in json["user"].items() if user_name in value
]

<IPython.core.display.Javascript object>

In [63]:
json.head()

,index,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,...,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers,user_name
0,0,https://twitter.com/LewisHowes/status/1351339921307545606,2021-01-19 01:25:18,@Arashdeep2002 Exactly! 💯,@Arashdeep2002 Exactly! 💯,1351339921307545606,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],0,...,1167437342069727233,en,"<a href=""http://www.zendesk.com"" rel=""nofollow"">Zendesk</a>",http://www.zendesk.com,Zendesk,None,NaN,None,"[{'username': 'Arashdeep2002', 'displayname': 'Arashdeep Kaur', 'id': 1344935179542908932, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/Arashdeep2002'}]",LewisHowes
1,1,https://twitter.com/LewisHowes/status/1351313050159222786,2021-01-18 23:38:31,@EDGcoach Thank you!,@EDGcoach Thank you!,1351313050159222786,"{'username': 'LewisHowes', 'displayname': 'Lewis Howes', 'id': 14777649, 'description': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'rawDescription': 'NY Times Bestselling author. Top Podcast with 300 million downloads. Former pro football current @usath text me here and stay connected! 📲 614-350-3960', 'descriptionUrls': [], 'verified': True, 'created': '2008-05-14T19:54:36+00:00', 'followersCount': 193432, 'friendsCount': 34018, 'statusesCount': 131994, 'favouritesCount': 152146, 'listedCount': 5431, 'mediaCount': 4230, 'location': 'Ohio boy in NYC and LA', 'protected': False, 'linkUrl': 'http://lewishowes.com', 'linkTcourl': 'https://t.co/hLkL1LJ3d4', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/904400279113687041/bSzT-5v4_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/14777649/1600464122', 'url': 'https://twitter.com/LewisHowes'}",[],[],1,...,1350507769602965505,en,"<a href=""http://www.zendesk.com"" rel=""nofollow"">Zendesk</a>",http://www.zendesk.com,Zendesk,None,NaN,None,"[{'username': 'EDGcoach', 'displayname': 'Empowering Inferno', 'id': 764510880822235136, 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'url': 'https://twitter.com/EDGcoach'}]",LewisHowes
2,2,https://twitter.com/LewisHowes/status/1351304297653350400,2021-01-18 23:03:45,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,@ogony_john Indeed! Thanks a lot for the tweet! 🙌,1351304297653350400,"{'u

<IPython.core.display.Javascript object>